# NOTE
Plotting logic has been moved into `cross_sensor_cal.sensor_panel_plots`
(`make_sensor_vs_neon_panels`, `make_micasense_vs_landsat_panels`).
This notebook is kept only for historical reference.


In [7]:
pip install spectral

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.0/249.0 kB 8.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install -e . 

Obtaining file:///home/jovyan/data-store/cross-sensor-cal
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for cross_sensor_cal (pyproject.toml) ... done
  Created wheel for cross_sensor_cal: filename=cross_sensor_cal-0.1.0-0.editable-py3-none-any.whl size=27465 sha256=a33dfdffd2252222c13d8bfdcdce70cc02410081951ab420422fc02a3771bf2a
  Stored in directory: /tmp/pip-ephem-wheel-cache-adjwenku/wheels/18/70/06/063f1fc66efdafe44297ee2e506086a9ff3bc04776dd2148a3
Successfully built cross_sensor_cal
  Attempting uninstall: cross_sensor_cal
    Found existing installation: cross_sensor_cal 0.1.0
    Uninstalling cross_sensor_cal-0.1.0:
      Successfully uninstalled cross_sensor_cal-0.1.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 MB 109.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 138.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
from src.polygon_extraction import control_function_for_extraction
from pathlib import Path


ModuleNotFoundError: No module named 'spectral'

In [76]:
# Example usage
directory = Path("Convolution_Reflectance_Resample_Landsat_5_TM")  # or wherever your ENVI files live
#polygon_path = Path("/home/jovyan/data-store/polygons/site_polygons.gpkg")

control_function_for_extraction(directory, None)


[DEBUG] Writing to full_extracted_pixels/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_Landsat_5_TM_envi_spectral_data.parquet
[INFO] Processing NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_Landsat_5_TM_envi.img with 6 bands as ENVI_band_


Processing NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_Landsat_5_TM_envi.img: 100%|██████████| 181/181 [00:10<00:00, 17.50chunk/s]


In [9]:
from pathlib import Path
import re
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

# band names like ENVI_band_417 / Masked_band_12 / Original_band_3
_BAND_RE = re.compile(r"^(ENVI|Masked|Original)_band_(\d+)$")

def _leading_band_columns(colnames):
    """
    Return the leading run of band columns (in order as stored),
    stopping at the first non-band column.
    """
    bands = []
    for c in colnames:
        m = _BAND_RE.match(c)
        if m:
            bands.append(c)
        else:
            break
    return bands

def _band_indices(band_cols):
    idxs = []
    for c in band_cols:
        m = _BAND_RE.match(c)
        if m:
            idxs.append(int(m.group(2)))
    return idxs

def validate_bands_in_dir(
    directory,
    expected_bands: int = 426,
    sample_rows: int = 4000,
    recursive: bool = True,
    value_range: tuple | None = (0.0, 1.0),  # set None to skip; or (0, 10000) if scaled reflectance
    eps_all_zero: float = 1e-12,
    show_preview: bool = True,
    meta_cols = ("Raster_File","CRS","Chunk_Number","Pixel_X","Pixel_Y"),
):
    """
    For each Parquet file in `directory`:
      - Identify leading band columns (assumes bands come first).
      - Check band count == expected_bands.
      - Sample values to test: finite, not-all-zero per band, and optional range.
      - Print a compact summary and return a DataFrame with results.
    """
    directory = Path(directory)
    files = sorted(directory.rglob("*.parquet") if recursive else directory.glob("*.parquet"))
    if not files:
        print(f"[INFO] No .parquet files in {directory}")
        return pd.DataFrame()

    pd.set_option("display.max_colwidth", None)
    pd.set_option("display.width", None)
    pd.set_option("display.max_columns", None)

    rows = []
    for f in files:
        rec = {
            "file": str(f),
            "ok_open": False,
            "num_row_groups": None,
            "num_rows_meta": None,
            "band_count": None,
            "first_cols_head": None,
            "first_band": None,
            "last_band": None,
            "band_index_min": None,
            "band_index_max": None,
            "bands_ok": None,
            "finite_ok": None,
            "all_zero_bands": [],
            "range_fail_bands": [],
            "error": None,
        }
        try:
            pf = pq.ParquetFile(f)
            rec["ok_open"] = True
            rec["num_row_groups"] = pf.num_row_groups
            rec["num_rows_meta"] = pf.metadata.num_rows if pf.metadata else None

            schema_cols = pf.schema_arrow.names
            rec["first_cols_head"] = schema_cols[:8]

            # take bands as the LEADING run
            band_cols = _leading_band_columns(schema_cols)
            rec["band_count"] = len(band_cols)

            # compute band index stats
            idxs = _band_indices(band_cols)
            if idxs:
                rec["band_index_min"] = int(np.min(idxs))
                rec["band_index_max"] = int(np.max(idxs))
            rec["first_band"] = band_cols[0] if band_cols else None
            rec["last_band"]  = band_cols[-1] if band_cols else None

            # expectation
            rec["bands_ok"] = (rec["band_count"] == expected_bands)

            # Read a small sample from the first row group (fast) for value checks
            cols_to_read = band_cols + [c for c in meta_cols if c in schema_cols]
            table = pf.read_row_group(0, columns=cols_to_read)
            df = table.to_pandas()
            if len(df) > sample_rows:
                df = df.head(sample_rows)
            band_df = df[band_cols].astype("float32") if band_cols else pd.DataFrame()

            # finite check
            if not band_df.empty:
                finite_ok = np.isfinite(band_df.to_numpy()).all()
            else:
                finite_ok = False
            rec["finite_ok"] = bool(finite_ok)

            # not all zero per band
            all_zero_bands = []
            if not band_df.empty:
                # max abs per band
                max_abs = band_df.abs().max(axis=0)
                for c, m in max_abs.items():
                    if not (m > eps_all_zero):
                        all_zero_bands.append(c)
            rec["all_zero_bands"] = all_zero_bands

            # range check (optional)
            range_fail_bands = []
            if value_range is not None and not band_df.empty:
                lo, hi = value_range
                # check band-wise if ANY sampled value is out of range
                too_low  = (band_df < lo).any(axis=0)
                too_high = (band_df > hi).any(axis=0)
                for c in band_cols:
                    if (c in too_low.index and bool(too_low[c])) or (c in too_high.index and bool(too_high[c])):
                        range_fail_bands.append(c)
            rec["range_fail_bands"] = range_fail_bands

            # Optional tiny preview for human eyes
            if show_preview:
                print(f"\n=== {f.name} ===")
                print(f"leading band cols: {rec['band_count']} (expected {expected_bands}) | "
                      f"first: {rec['first_band']} | last: {rec['last_band']} | "
                      f"indices: [{rec['band_index_min']}, {rec['band_index_max']}] | "
                      f"row_groups: {rec['num_row_groups']} | rows(meta): {rec['num_rows_meta']}")
                if not df.empty:
                    preview_cols = [c for c in meta_cols if c in df.columns] + band_cols[:5]
                    display(df[preview_cols].head(1))

        except Exception as e:
            rec["error"] = str(e)

        rows.append(rec)

    out = pd.DataFrame(rows)

    # Directory-level summary
    print("\n===== BAND LAYOUT VALIDATION =====")
    print(f"Directory: {directory}")
    print(f"Files: {len(out)}")
    print(f"Open errors: {(out['ok_open'] == False).sum()}")
    print(f"Band-count mismatches: {(out['bands_ok'] == False).sum()} (expected {expected_bands})")
    print(f"Finite check failures: {(out['finite_ok'] == False).sum()}")
    print(f"Files with all-zero bands (any): {(out['all_zero_bands'].apply(lambda x: len(x) > 0)).sum()}")
    if value_range is not None:
        print(f"Range check failures (any band outside {value_range}): {(out['range_fail_bands'].apply(lambda x: len(x) > 0)).sum()}")

    # Compact view
    compact = out[[
        "file","band_count","bands_ok","first_band","last_band",
        "band_index_min","band_index_max","num_row_groups","num_rows_meta",
        "finite_ok","all_zero_bands","range_fail_bands","error"
    ]].copy()
    display(compact.head(50))
    return out

# For reflectance 0–1:
_ = validate_bands_in_dir(
    "full_extracted_pixels",
    expected_bands=426,
    value_range=(0.0, 10000.0),
    sample_rows=4000,
    recursive=True,
    show_preview=True
)

# If your reflectance is scaled (e.g., 0–10000), change the range:
# _ = validate_bands_in_dir("/home/jovyan/data-store/output_parquet", expected_bands=426, value_range=(0, 10000))



=== NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_reflectance_brdfandtopo_corrected_envi_spectral_data.parquet ===
leading band cols: 426 (expected 426) | first: ENVI_band_1 | last: ENVI_band_426 | indices: [1, 426] | row_groups: 181 | rows(meta): 11151207


,Raster_File,CRS,Chunk_Number,Pixel_X,Pixel_Y,ENVI_band_1,ENVI_band_2,ENVI_band_3,ENVI_band_4,ENVI_band_5
0,NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_reflectance_brdfandtopo_corrected_envi.img,EPSG:32613,0,453780.0,4436611.0,78.0,80.0,118.0,69.0,56.265697



=== NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_Landsat_5_TM_envi_spectral_data.parquet ===
leading band cols: 6 (expected 426) | first: ENVI_band_1 | last: ENVI_band_6 | indices: [1, 6] | row_groups: 181 | rows(meta): 11151207


,Raster_File,CRS,Chunk_Number,Pixel_X,Pixel_Y,ENVI_band_1,ENVI_band_2,ENVI_band_3,ENVI_band_4,ENVI_band_5
0,NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_Landsat_5_TM_envi.img,EPSG:32613,0,453780.0,4436611.0,61.69389,82.067566,192.176132,716.169678,191.106964



=== NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_Landsat_7_ETM+_envi_spectral_data.parquet ===
leading band cols: 6 (expected 426) | first: ENVI_band_1 | last: ENVI_band_6 | indices: [1, 6] | row_groups: 181 | rows(meta): 11151207


...

,Raster_File,CRS,Chunk_Number,Pixel_X,Pixel_Y,ENVI_band_1,ENVI_band_2,ENVI_band_3,ENVI_band_4
0,NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_MicaSense-to-match_TM_and_ETM+_envi.img,EPSG:32613,0,453780.0,4436611.0,43.976063,102.227814,39.157265,818.162598



=== NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_MicaSense_envi_spectral_data.parquet ===
leading band cols: 10 (expected 426) | first: ENVI_band_1 | last: ENVI_band_10 | indices: [1, 10] | row_groups: 181 | rows(meta): 11151207


,Raster_File,CRS,Chunk_Number,Pixel_X,Pixel_Y,ENVI_band_1,ENVI_band_2,ENVI_band_3,ENVI_band_4,ENVI_band_5
0,NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_MicaSense_envi.img,EPSG:32613,0,453780.0,4436611.0,50.587128,43.976063,88.937439,102.227814,53.009937



===== BAND LAYOUT VALIDATION =====
Directory: full_extracted_pixels
Files: 8
Open errors: 0
Band-count mismatches: 7 (expected 426)
Finite check failures: 0
Files with all-zero bands (any): 0
Range check failures (any band outside (0.0, 10000.0)): 1


,file,band_count,bands_ok,first_band,last_band,band_index_min,band_index_max,num_row_groups,num_rows_meta,finite_ok,all_zero_bands,range_fail_bands,error
0,full_extracted_pixels/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_reflectance_brdfandtopo_corrected_envi_spectral_data.parquet,426,True,ENVI_band_1,ENVI_band_426,1,426,181,11151207,True,[],"[ENVI_band_58, ENVI_band_61]",None
1,full_extracted_pixels/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_Landsat_5_TM_envi_spectral_data.parquet,6,False,ENVI_band_1,ENVI_band_6,1,6,181,11151207,True,[],[],None
2,full_extracted_pixels/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_Landsat_7_ETM+_envi_spectral_data.parquet,6,False,ENVI_band_1,ENVI_band_6,1,6,181,11151207,True,[],[],None
3,full_extracted_pixels/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_Landsat_8_OLI_envi_spectral_data.parquet,7,False,ENVI_band_1,ENVI_band_7,1,7,181,11151207,True,[],[],None
4,full_extracted_pixels/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_Landsat_9_OLI-2_envi_spectral_data.parquet,7,False,ENVI_band_1,ENVI_band_7,1,7,181,11151207,True,[],[],None
5,full_extracted_pixels/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_MicaSense-to-match_OLI_and_OLI-2_envi_spectral_data.parquet,5,False,ENVI_band_1,ENVI_band_5,1,5,181,11151207,True,[],[],None
6,full_extracted_pixels/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_MicaSense-to-match_TM_and_ETM+_envi_spectral_data.parquet,4,False,ENVI_band_1,ENVI_band_4,1,4,181,11151207,True,[],[],None
7,full_extracted_pixels/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_MicaSense_envi_spectral_data.parquet,10,False,ENVI_band_1,ENVI_band_10,1,10,181,11151207,True,[],[],None


In [7]:
# Re-define without importing pyarrow at module import time (to avoid errors in this sandbox).
# We'll import pyarrow inside the function at call-time in the user's environment.

from pathlib import Path
import re
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

_BAND_RE = re.compile(r"^(ENVI|Masked|Original)_band_(\d+)$")

def _leading_band_columns_sorted(colnames):
    bands = []
    for c in colnames:
        m = _BAND_RE.match(c)
        if m:
            bands.append((c, int(m.group(2))))
        else:
            break
    bands.sort(key=lambda x: x[1])
    return [c for c, _ in bands], [i for _, i in bands]

def plot_spectra_from_parquet_dir(
    directory,
    output_dir: str | None = None,
    recursive: bool = True,
    max_lines_per_file: int = 5000,
    median_sample_rows: int = 50000,
    sentinel_at_or_below: float = -9990,
    mask_integer_lo: int = 1,
    mask_integer_hi: int = 255,
    dpi: int = 200
):
    """
    For each Parquet file in `directory`, create a PNG plot:
      - X-axis: band index
      - Y-axis: reflectance
      - One line per pixel (subsampled to at most `max_lines_per_file`)
      - A median (50th percentile) trendline across a uniform sample of pixels

    Notes:
      - Imports `pyarrow.parquet` at call-time. Ensure `pyarrow` is installed in your environment.
    """
    try:
        import pyarrow.parquet as pq  # imported here to avoid environment issues in this sandbox
    except Exception as e:
        raise ImportError("This function requires 'pyarrow'. Please install it (e.g., `pip install pyarrow`).") from e

    directory = Path(directory)
    out_dir = Path(output_dir) if output_dir else (directory / "_spectral_plots")
    out_dir.mkdir(parents=True, exist_ok=True)

    files = sorted(directory.rglob("*.parquet") if recursive else directory.glob("*.parquet"))
    if not files:
        print(f"[INFO] No .parquet files found under: {directory}")
        return []

    generated = []
    rng = np.random.default_rng(42)

    for f in files:
        try:
            pf = pq.ParquetFile(f)
        except Exception as e:
            print(f"[WARN] Skipping {f.name}: cannot open ({e})")
            continue

        cols = pf.schema_arrow.names
        band_cols, band_idx = _leading_band_columns_sorted(cols)
        if not band_cols:
            print(f"[WARN] {f.name}: no leading band columns found; skipping.")
            continue

        total_rows = pf.metadata.num_rows if pf.metadata and pf.metadata.num_rows is not None else None
        if total_rows is None or total_rows == 0:
            print(f"[WARN] {f.name}: no rows; skipping.")
            continue
        stride = max(1, math.ceil(total_rows / max_lines_per_file)) if max_lines_per_file else 1
        p = min(1.0, median_sample_rows / total_rows) if median_sample_rows else 1.0

        fig = plt.figure(figsize=(10, 6))
        ax = plt.gca()

        x = np.array(band_idx, dtype=np.float32)

        samples = []
        plotted = 0
        processed_rows = 0

        for rg in range(pf.num_row_groups):
            try:
                table = pf.read_row_group(rg, columns=band_cols)
            except Exception as e:
                print(f"[WARN] {f.name}: failed to read row group {rg} ({e}); continuing.")
                continue

            df = table.to_pandas()
            arr = df.to_numpy(dtype=np.float32, copy=False)

            # Missing-value masks
            arr = np.where(arr <= sentinel_at_or_below, np.nan, arr)
            int_mask = (arr >= mask_integer_lo) & (arr <= mask_integer_hi) & (arr == np.floor(arr))
            arr = np.where(int_mask, np.nan, arr)

            rows_in_rg = arr.shape[0]

            if max_lines_per_file:
                idxs = np.arange(rows_in_rg)
                plot_mask = ((idxs + processed_rows) % stride == 0)
                plot_rows = arr[plot_mask]
                for row in plot_rows:
                    if np.all(np.isnan(row)):
                        continue
                    ax.plot(x, row, linewidth=0.5, alpha=0.15)  # no explicit color
                    plotted += 1

            if median_sample_rows:
                if p >= 1.0:
                    take = arr
                else:
                    k = int(np.ceil(rows_in_rg * p))
                    if k > 0 and rows_in_rg > 0:
                        sel = rng.choice(rows_in_rg, size=min(k, rows_in_rg), replace=False)
                        take = arr[sel]
                    else:
                        take = None
                if take is not None and take.size > 0:
                    samples.append(take)

            processed_rows += rows_in_rg

        if median_sample_rows and samples:
            sample_arr = np.vstack(samples)
            if sample_arr.shape[0] > median_sample_rows:
                sel = rng.choice(sample_arr.shape[0], size=median_sample_rows, replace=False)
                sample_arr = sample_arr[sel]
            med = np.nanmedian(sample_arr, axis=0)
            ax.plot(x, med, linewidth=2.0, alpha=1.0, label="Median (50th)")
            ax.legend()

        ax.set_xlabel("Band index")
        ax.set_ylabel("Reflectance")
        ax.set_title(f.name)
        fig.tight_layout()

        out_path = out_dir / f"{f.stem}_spectra.png"
        try:
            fig.savefig(out_path, dpi=200, bbox_inches="tight")
            generated.append(out_path)
        except Exception as e:
            print(f"[WARN] Failed to save plot for {f.name}: {e}")
        finally:
            plt.close(fig)

        print(f"[OK] {f.name}: plotted ~{plotted} pixel lines -> {out_path}")

    return generated

# Example run (adjust the path)
plots = plot_spectra_from_parquet_dir("full_extracted_pixels")

# Or customize:
# plots = plot_spectra_from_parquet_dir(
#     "/home/jovyan/data-store/output_parquet",
#     output_dir="/home/jovyan/data-store/output_plots",
#     max_lines_per_file=8000,        # more/less pixel lines on each plot
#     median_sample_rows=100000,      # sample size for the 50th-percentile trend line
#     sentinel_at_or_below=-9990,     # treat <= -9990 (e.g., -9999) as missing
#     mask_integer_lo=1, mask_integer_hi=255  # palette-like integers treated as missing
# )


[OK] NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_reflectance_brdfandtopo_corrected_envi_spectral_data.parquet: plotted ~4999 pixel lines -> full_extracted_pixels/_spectral_plots/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_reflectance_brdfandtopo_corrected_envi_spectral_data_spectra.png
[OK] NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_Landsat_5_TM_envi_spectral_data.parquet: plotted ~4999 pixel lines -> full_extracted_pixels/_spectral_plots/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_Landsat_5_TM_envi_spectral_data_spectra.png
[OK] NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_Landsat_7_ETM+_envi_spectral_data.parquet: plotted ~4999 pixel lines -> full_extracted_pixels/_spectral_plots/NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_Landsat_7_ETM+_envi_spectral_data_spectra.png
[OK] NEON_D13_NIWO_DP1.30006.001_L020-1_20230815_directional_resampled_Landsat_8_OLI_envi_spectral_data.par

In [14]:
from pathlib import Path
import re
import duckdb

# --- patterns ---
# old broad prefix matcher (kept as fallback)
_GROUP_RE = re.compile(r'^(?P<prefix>.+?_L\d{3}-\d_\d{8}_)')

# canonical key: NEON_<domain>_<site>_DP1[.product]?_L###-#_<YYYYMMDD>_
_CANON_RE = re.compile(
    r'^(?P<site>NEON_[A-Z0-9]+_[A-Z0-9]+_DP1)(?:\.\d+\.\d+)?_(?P<tile>L\d{3}-\d)_(?P<date>\d{8})_'
)

def _canonical_group_key(fname: str) -> str | None:
    m = _CANON_RE.match(fname)
    if m:
        return f"{m.group('site')}_{m.group('tile')}_{m.group('date')}_"
    m2 = _GROUP_RE.match(fname)
    return m2.group('prefix') if m2 else None

# sensor/source label recognizer (order matters)
def _label_from_name(name: str) -> str:
    s = name
    if "resampled_Landsat_9_OLI-2" in s:   return "Landsat_9_OLI-2"
    if "resampled_Landsat_8_OLI" in s:     return "Landsat_8_OLI"
    if "resampled_Landsat_7_ETM+" in s:    return "Landsat_7_ETM+"
    if "resampled_Landsat_5_TM" in s:      return "Landsat_5_TM"
    if "resampled_MicaSense-to-match_OLI_and_OLI-2" in s: return "MicaSense_to-match_OLI_and_OLI-2"
    if "resampled_MicaSense-to-match_TM_and_ETM+" in s:   return "MicaSense_to-match_TM_and_ETM+"
    if "resampled_MicaSense_envi" in s or "resampled_MicaSense" in s: return "MicaSense"
    if "directional_reflectance_brdfandtopo_corrected_envi" in s: return "NEON_directional_BRDFTopo"
    if "reflectance_brdfandtopo_corrected_envi" in s:              return "NEON_BRDFTopo"
    return "Unknown"

# band column pattern
_BAND_RE = re.compile(r"^(ENVI|Masked|Original)_band_(\d+)$")

def _quote_ident(col: str) -> str:
    return '"' + col.replace('"', '""') + '"'

def _band_cols_duckdb(con: duckdb.DuckDBPyConnection, parquet_path: Path) -> list[str]:
    rows = con.execute(f"DESCRIBE SELECT * FROM read_parquet('{parquet_path.as_posix()}')").fetchall()
    names = [r[0] for r in rows]
    return [n for n in names if _BAND_RE.match(n)]

def merge_matching_parquet_sets(
    input_dir,
    output_dir=None,
    join_on=("Pixel_ID",),
    keep_meta=("Pixel_ID","Pixel_X","Pixel_Y","CRS"),
    expected_big=426,
    verbose=True
):
    """
    Merge sets of parquet files that share the canonical prefix:
      NEON_<domain>_<site>_DP1[.product]?_L###-#_<YYYYMMDD>_*

    - Picks a 426-band file as base when available.
    - LEFT JOINs all other files on Pixel_ID (fallback Pixel_X,Pixel_Y).
    - Renames bands to <LABEL>_band_<n> and SELECTs them (fixes previous t0.*-only bug).
    - Writes one merged parquet per set into a sibling "<dir>_merged/" (or custom output_dir).
    """
    input_dir = Path(input_dir)
    if output_dir is None:
        output_dir = input_dir.parent / f"{input_dir.name}_merged"
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    con = duckdb.connect()
    files = sorted(input_dir.glob("*.parquet"))

    # group files by canonical key
    groups: dict[str, list[Path]] = {}
    for f in files:
        key = _canonical_group_key(f.name)
        if not key:
            if verbose:
                print(f"[SKIP] Unrecognized name (no group key): {f.name}")
            continue
        groups.setdefault(key, []).append(f)

    if not groups:
        print(f"[INFO] No matching sets found under: {input_dir}")
        con.close()
        return []

    written = []

    for key, paths in sorted(groups.items()):
        # pick base (prefer largest band count)
        band_counts = {}
        for p in paths:
            try:
                band_counts[p] = len(_band_cols_duckdb(con, p))
            except Exception:
                band_counts[p] = 0

        base_path = sorted(paths, key=lambda p: (-band_counts.get(p, 0), p.name))[0]
        base_label = _label_from_name(base_path.name)
        base_bands = _band_cols_duckdb(con, base_path)

        # determine join cols
        rows = con.execute(f"DESCRIBE SELECT * FROM read_parquet('{base_path.as_posix()}')").fetchall()
        base_cols = {r[0] for r in rows}
        join_cols = list(join_on)
        if not all(c in base_cols for c in join_cols):
            if {"Pixel_X","Pixel_Y"}.issubset(base_cols):
                join_cols = ["Pixel_X","Pixel_Y"]
            else:
                raise RuntimeError(f"Base {base_path.name} missing join cols; has: {sorted(list(base_cols))[:20]}")

        # build base CTE (rename its bands)
        base_select = []
        for c in keep_meta:
            if c in base_cols:
                base_select.append(_quote_ident(c))
        for b in base_bands:
            m = _BAND_RE.match(b)
            idx = m.group(2) if m else b
            newname = f"{base_label}_band_{idx}"
            base_select.append(f"{_quote_ident(b)} AS {_quote_ident(newname)}")

        base_cte_name = "t0"
        base_cte = f"""{base_cte_name} AS (
            SELECT {", ".join(base_select)}
            FROM read_parquet('{base_path.as_posix()}')
        )"""

        # other CTEs + joins
        ctes = [base_cte]
        join_clauses = []
        select_list = [f"{base_cte_name}.*"]  # start with all base cols

        t_idx = 1
        for p in sorted(paths):
            if p == base_path:
                continue

            label = _label_from_name(p.name)
            bcols = _band_cols_duckdb(con, p)
            if not bcols:
                if verbose:
                    print(f"[WARN] {p.name}: no band columns, skipping.")
                continue

            rows = con.execute(f"DESCRIBE SELECT * FROM read_parquet('{p.as_posix()}')").fetchall()
            cols = {r[0] for r in rows}

            # ensure join cols exist
            if not all(c in cols for c in join_cols):
                if verbose:
                    print(f"[WARN] {p.name}: missing join columns {join_cols}, skipping.")
                continue

            # project join cols + renamed bands
            sel_parts = [ _quote_ident(c) for c in join_cols ]
            renamed_cols = []  # keep the new names so we can SELECT them later
            for b in bcols:
                m = _BAND_RE.match(b)
                idx = m.group(2) if m else b
                newname = f"{label}_band_{idx}"
                sel_parts.append(f"{_quote_ident(b)} AS {_quote_ident(newname)}")
                renamed_cols.append(newname)

            ti = f"t{t_idx}"
            ctes.append(f"""{ti} AS (
                SELECT {", ".join(sel_parts)}
                FROM read_parquet('{p.as_posix()}')
            )""")

            # LEFT JOIN USING (join_cols)
            using_clause = ", ".join(_quote_ident(c) for c in join_cols)
            join_clauses.append(f"LEFT JOIN {ti} USING ({using_clause})")

            # **FIX**: actually SELECT the joined sensor columns
            select_list.extend(f'{ti}.{_quote_ident(c)}' for c in renamed_cols)

            t_idx += 1

        # write output
        output_name = key.rstrip("_") + "_merged.parquet"
        out_path = output_dir / output_name

        with_clause = "WITH " + ",\n".join(ctes)
        joins = "\n".join(join_clauses)
        final_sql = f"""{with_clause}
        SELECT {", ".join(select_list)}
        FROM {base_cte_name}
        {joins}
        """

        con.execute(f"COPY ({final_sql}) TO '{out_path.as_posix()}' (FORMAT PARQUET)")

        if verbose:
            big_count = len(base_bands)
            if big_count != expected_big:
                print(f"[INFO] {base_path.name}: base bands={big_count} (expected ~{expected_big})")
            print(f"[OK] Wrote {out_path} from {len(paths)} files; join keys: {join_cols}")

        written.append(out_path)

    con.close()
    return written

outs = merge_matching_parquet_sets("full_extracted_pixels", verbose=True)

# Preview the merged columns for one output:
import duckdb, pathlib
con = duckdb.connect()
merged = pathlib.Path("full_extracted_pixels_merged")
p = sorted(merged.glob("*.parquet"))[0]
con.sql(f"DESCRIBE SELECT * FROM read_parquet('{p.as_posix()}')").df()[["column_name","column_type"]]



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[OK] Wrote full_extracted_pixels_merged/NEON_D13_NIWO_DP1_L020-1_20230815_merged.parquet from 8 files; join keys: ['Pixel_ID']


,column_name,column_type
0,Pixel_ID,BIGINT
1,Pixel_X,DOUBLE
2,Pixel_Y,DOUBLE
3,CRS,VARCHAR
4,NEON_directional_BRDFTopo_band_1,FLOAT
...,...,...
470,MicaSense_band_6,FLOAT
471,MicaSense_band_7,FLOAT
472,MicaSense_band_8,FLOAT
473,MicaSense_band_9,FLOAT


In [12]:
"""
Notebook-friendly module: make panel plots comparing corrected NEON 426-band spectra to
resampled sensor bands from *merged* Parquet files. One PNG per merged Parquet, saved to a
sibling folder (same level as the input directory).

Layout
- One row per sensor
- Within each row, subplots are ordered left→right by band index (band 1 at far left)
- Columns = max number of plotted bands across sensors (unused cells are turned off)

Key features
- Auto-detects the corrected set (NEON_BRDFTopo or NEON_directional_BRDFTopo)
- Auto-detects available sensors (or you can require specific ones)
- Skips palette-like columns using a per-column integer-fraction threshold
- Excludes rows with non-zero *_error flags (auto-detected patterns)
- Excludes negative values before regression
- Samples rows per band-pair with DuckDB for speed/memory safety
- Adds OLS regression per subplot with dashed fit line and equation + R² + n
- Labels bands with real wavelength/FWHM values from JSONs

Usage in a notebook
-------------------
from sensor_panel_plots_notebook import make_all_sensor_panels
plots = make_all_sensor_panels(
    "full_extracted_pixels_merged",
)
len(plots), plots[:3]

Requires:  pip install duckdb
"""

from pathlib import Path
import re
import json
from typing import Dict, List, Tuple, Optional

import duckdb
import numpy as np
import matplotlib.pyplot as plt

# ---------- naming helpers ----------
_BAND_LABEL_RE = re.compile(r"^(?P<label>.+)_band_(?P<idx>\d+)$")
CORRECTED_CANDIDATES = ("NEON_BRDFTopo", "NEON_directional_BRDFTopo")

# Expected band counts for typical resampled sets (informational)
DEFAULT_EXPECTED_SETS: Dict[str, int] = {
    "Landsat_5_TM": 6,
    "Landsat_7_ETM+": 6,
    "Landsat_8_OLI": 7,
    "Landsat_9_OLI-2": 7,
    "MicaSense": 10,
    "MicaSense_to-match_OLI_and_OLI-2": 5,
    "MicaSense_to-match_TM_and_ETM+": 4,
}

# ---------- real band centers / FWHMs from your JSONs ----------
_HYPERSPEC_JSON = Path("data/hyperspectral_bands.json")
_LANDSAT_PARAMS_JSON = Path("data/landsat_band_parameters.json")

with _HYPERSPEC_JSON.open("r") as f:
    _HYPERSPEC_BANDS = list(json.load(f)["bands"])  # length 426, values in nm

with _LANDSAT_PARAMS_JSON.open("r") as f:
    _SENSOR_PARAMS = json.load(f)

# Map our column labels -> keys in the params JSON
_PARAM_KEY_ALIASES = {
    "Landsat_5_TM": "Landsat 5 TM",
    "Landsat_7_ETM+": "Landsat 7 ETM+",
    "Landsat_8_OLI": "Landsat 8 OLI",
    "Landsat_9_OLI-2": "Landsat 9 OLI-2",
    "MicaSense": "MicaSense",
    "MicaSense_to-match_TM_and_ETM+": "MicaSense-to-match TM and ETM+",
    "MicaSense_to-match_OLI_and_OLI-2": "MicaSense-to-match OLI and OLI-2",
}

# ------------------------ helpers ------------------------

def _sensor_band_text(sensor_label: str, band_idx1: int) -> str:
    """Return a nice label with wavelength/FWHM if known."""
    key = _PARAM_KEY_ALIASES.get(sensor_label)
    if key and key in _SENSOR_PARAMS:
        wl = _SENSOR_PARAMS[key].get("wavelengths", [])
        fw = _SENSOR_PARAMS[key].get("fwhms", [])
        if 1 <= band_idx1 <= len(wl):
            fwhm = fw[band_idx1-1] if band_idx1-1 < len(fw) else None
            if fwhm is not None:
                return f"{sensor_label} B{band_idx1} ({wl[band_idx1-1]:g} nm ±{fwhm:g})"
            return f"{sensor_label} B{band_idx1} ({wl[band_idx1-1]:g} nm)"
    return f"{sensor_label} band {band_idx1}"


def _corrected_band_text(corrected_label: str, band_idx1: int) -> str:
    """Return corrected set label including NEON band center if known."""
    if 1 <= band_idx1 <= len(_HYPERSPEC_BANDS):
        return f"{corrected_label} {band_idx1} ({_HYPERSPEC_BANDS[band_idx1-1]:.1f} nm)"
    return f"{corrected_label} band {band_idx1}"


def _read_schema(con: duckdb.DuckDBPyConnection, parquet_path: Path) -> List[str]:
    """
    Robustly return *flat, numeric* column names from a Parquet file.
    Order of attempts:
      1) DuckDB DESCRIBE of read_parquet
      2) DuckDB parquet_schema() table function
      3) PyArrow schema inspection (fallback of last resort)
    We drop LIST/STRUCT/MAP/DICT columns and keep only numeric scalars.
    """
    # 1) DuckDB DESCRIBE
    try:
        rows = con.sql(
            f"DESCRIBE SELECT * FROM read_parquet('{parquet_path.as_posix()}')"
        ).fetchall()
        return [r[0] for r in rows]
    except Exception:
        pass

    # 2) DuckDB parquet_schema()
    try:
        sch = con.sql(
            f"SELECT name, type FROM parquet_schema('{parquet_path.as_posix()}')"
        ).fetchall()
        flat_cols = [
            name for name, typ in sch
            if isinstance(name, str)
            and isinstance(typ, str)
            and not any(t in typ.upper() for t in ("LIST", "STRUCT", "MAP", "UNION", "DICTIONARY"))
        ]
        if flat_cols:
            return flat_cols
    except Exception:
        pass

    # 3) PyArrow fallback
    try:
        import pyarrow as pa
        import pyarrow.parquet as pq
        pf = pq.ParquetFile(parquet_path.as_posix())
        schema = pf.schema_arrow
        def _is_flat_numeric(t: 'pa.DataType') -> bool:
            return pa.types.is_integer(t) or pa.types.is_floating(t)
        cols = [f.name for f in schema if _is_flat_numeric(f.type)]
        if cols:
            return cols
    except Exception:
        pass

    return []


def _band_map(columns: List[str]) -> Dict[str, List[int]]:
    """Return mapping: label -> sorted list of band indices for '<label>_band_<idx>' columns."""
    out: Dict[str, List[int]] = {}
    for c in columns:
        m = _BAND_LABEL_RE.match(c)
        if m:
            out.setdefault(m.group("label"), []).append(int(m.group("idx")))
    for k in out:
        out[k] = sorted(out[k])
    return out


def _pick_corrected_label(bandmap: Dict[str, List[int]], need: int = 426) -> Optional[str]:
    for cand in CORRECTED_CANDIDATES:
        if cand in bandmap and len(bandmap[cand]) >= need:
            return cand
    return None


# Error flag helpers: treat non-zero as error (exclude)

def _possible_error_cols(label: str, idx: int) -> List[str]:
    i = str(idx)
    return [
        f"{label}_band_{i}_error",
        f"{label}_band_{i}_err",
        f"{label}_error_band_{i}",
        f"{label}_err_band_{i}",
    ]


def _present_error_cols(all_cols: List[str], label: str, idx: int) -> List[str]:
    return [c for c in _possible_error_cols(label, idx) if c in all_cols]


def _palette_fraction_by_column(
    con: duckdb.DuckDBPyConnection,
    parquet_path: Path,
    colnames: List[str],
    lo: int = 1,
    hi: int = 255,
) -> Dict[str, float]:
    """Return {column -> fraction of integer palette-like values} via a single DuckDB query."""
    if not colnames:
        return {}
    terms = [
        f'SUM((("{c}" BETWEEN {lo} AND {hi}) AND "{c}" = floor("{c}")))*1.0 / NULLIF(COUNT("{c}"),0) AS "{c}"'
        for c in colnames
    ]
    sql = f"SELECT {', '.join(terms)} FROM read_parquet('{parquet_path.as_posix()}')"
    row = con.sql(sql).fetchone()
    return {c: (row[i] if row[i] is not None else 0.0) for i, c in enumerate(colnames)}


def _sample_pair_df(
    con: duckdb.DuckDBPyConnection,
    parquet_path: Path,
    xcol: str,
    ycol: str,
    max_points: int,
    sentinel_at_or_below: float,
    extra_zero_filters: Optional[List[str]] = None,
):
    """
    Filter (finite & > sentinel), optional '== 0' filters for error columns, then sample.
    Uses DuckDB's USING SAMPLE when available; falls back to ORDER BY random() LIMIT.
    Returns a pandas DataFrame with only non-negative x and y for regression/plotting.
    """
    extra = ""
    if extra_zero_filters:
        extra = " AND " + " AND ".join(f'"{c}" = 0' for c in extra_zero_filters)

    base = f'''
    WITH base AS (
      SELECT "{xcol}" AS x, "{ycol}" AS y
      FROM read_parquet('{parquet_path.as_posix()}')
      WHERE
        isfinite("{xcol}") AND isfinite("{ycol}")
        AND "{xcol}" > {sentinel_at_or_below} AND "{ycol}" > {sentinel_at_or_below}
        {extra}
    )
    '''
    q_sample = base + f"SELECT * FROM base USING SAMPLE {max_points} ROWS"
    try:
        df = con.sql(q_sample).df()
    except Exception:
        q_fallback = base + f"SELECT * FROM base ORDER BY random() LIMIT {max_points}"
        df = con.sql(q_fallback).df()
    # Exclude negatives before regression
    return df[(df["x"] >= 0) & (df["y"] >= 0)]


def _linreg(x: np.ndarray, y: np.ndarray) -> Tuple[float, float]:
    """Return slope, intercept for y ~ m x + b; robust to small samples."""
    if x.size < 2 or y.size < 2:
        return float("nan"), float("nan")
    m, b = np.polyfit(x, y, 1)
    return float(m), float(b)


# ------------------------ main entry ------------------------

def make_all_sensor_panels(
    merged_dir,
    required_sets: Optional[Tuple[str, ...]] = None,   # None => auto: include all present sensors (except corrected)
    expected_sets: Dict[str, int] = DEFAULT_EXPECTED_SETS,
    output_root: Optional[Path] = None,                # default: sibling "<dir>_compare_panels"
    max_points_per_pair: int = 50_000,                 # reservoir sample size per subplot
    sentinel_at_or_below: float = -9990,               # drop <= this (e.g., -9999)
    palette_int_range: Tuple[int, int] = (1, 255),     # palette-like integer scan range
    palette_col_frac_threshold: float = 0.95,          # mark a COLUMN palette-like if >= this fraction
    strict_require_expected_counts: bool = False,      # if True, skip sensor if count != expected
    show_ones_line: bool = True,
    log: bool = True,
):
    """
    For each merged parquet in `merged_dir`:
      - find corrected 426-band set
      - choose sensors (required sets or auto-detect from expected_sets)
      - mark columns as palette-like if >= palette_col_frac_threshold of integer values in [lo,hi]
      - skip bands whose corrected OR sensor column is palette-like
      - exclude rows where per-band error columns (if present) are non-zero
      - sample and scatter-plot each remaining band pair; panel arranged as rows= sensors, cols= bands (ascending)
      - draw 1:1 line (gray) and OLS regression line (red dashed) with y = mx + b, R², and n
      - write PNGs to a sibling folder "<merged_dir>_compare_panels" (or `output_root`)

    Returns: list of output PNG Paths.
    """
    merged_dir = Path(merged_dir)
    if output_root is None:
        output_root = merged_dir.parent / f"{merged_dir.name}_compare_panels"
    output_root = Path(output_root)
    output_root.mkdir(parents=True, exist_ok=True)

    con = duckdb.connect()
    parquet_files = sorted(merged_dir.glob("*.parquet"))
    if not parquet_files:
        if log:
            print(f"[INFO] No merged parquet files in {merged_dir}")
        con.close()
        return []

    outputs: List[Path] = []
    lo, hi = palette_int_range

    for p in parquet_files:
        cols = _read_schema(con, p)
        if not cols:
            if log:
                print(f"[WARN] {p.name}: no readable flat numeric columns; skipping.")
            continue
        bmap = _band_map(cols)

        corrected = _pick_corrected_label(bmap, need=426)
        if corrected is None:
            if log:
                print(f"[SKIP] {p.name}: missing corrected 426-band set {CORRECTED_CANDIDATES}.")
            continue

        # Pick sensors
        if required_sets is None:
            sensors = [lab for lab in expected_sets if (lab in bmap and lab != corrected)]
        else:
            sensors = [lab for lab in required_sets if (lab in bmap and lab != corrected)]
            missing = [lab for lab in (required_sets or []) if lab not in bmap]
            if missing and log:
                print(f"[WARN] {p.name}: missing sensors -> {missing}")
        if not sensors:
            if log:
                print(f"[SKIP] {p.name}: no known sensor sets found.")
            continue

        # Optional strict count check
        if strict_require_expected_counts:
            bad = []
            for lab in list(sensors):
                exp = expected_sets.get(lab)
                have = len(bmap.get(lab, []))
                if exp is not None and have != exp:
                    sensors.remove(lab)
                    bad.append((lab, have, exp))
            if bad and log:
                print(f"[WARN] {p.name}: dropped sensors (count mismatch): {bad}")
            if not sensors:
                if log:
                    print(f"[SKIP] {p.name}: no sensors passed expected-count check.")
                continue

        # Build needed columns for palette scan (all corrected & sensor bands used)
        need_cols: List[str] = []
        for lab in sensors:
            for idx in bmap[lab]:
                need_cols.append(f'{corrected}_band_{idx}')
                need_cols.append(f'{lab}_band_{idx}')
        need_cols = sorted(set(need_cols))

        # Compute palette-like fractions per column
        pal_frac = _palette_fraction_by_column(con, p, need_cols, lo=lo, hi=hi)

        # Choose band pairs grouped by sensor (skip palette-like columns); keep bands ascending
        # sensor_pairs: {sensor -> [(idx, xcol, ycol), ...] }
        sensor_pairs: Dict[str, List[Tuple[int, str, str]]] = {}
        dropped_bands: List[Tuple[str, int, float, float]] = []
        for lab in sensors:
            pairs: List[Tuple[int, str, str]] = []
            for idx in bmap[lab]:
                xcol = f'{corrected}_band_{idx}'
                ycol = f'{lab}_band_{idx}'
                if pal_frac.get(xcol, 0.0) >= palette_col_frac_threshold or pal_frac.get(ycol, 0.0) >= palette_col_frac_threshold:
                    dropped_bands.append((lab, idx, pal_frac.get(xcol, 0.0), pal_frac.get(ycol, 0.0)))
                    continue
                pairs.append((idx, xcol, ycol))
            # ensure ascending by band index
            pairs.sort(key=lambda t: t[0])
            if pairs:
                sensor_pairs[lab] = pairs

        if not sensor_pairs:
            if log:
                print(f"[SKIP] {p.name}: all bands looked palette-like at ≥ {palette_col_frac_threshold:.2f}.")
            continue

        # Layout: one row per sensor, columns = max band count among sensors
        sensors_in_panel = [s for s in sensors if s in sensor_pairs]
        n_rows = len(sensors_in_panel)
        n_cols = max(len(sensor_pairs[s]) for s in sensors_in_panel)

        fig, axes = plt.subplots(
            n_rows, n_cols, figsize=(4.0*n_cols, 3.6*n_rows), squeeze=False
        )

        # Plot each row (sensor), columns ordered by band idx
        for r, lab in enumerate(sensors_in_panel):
            pairs = sensor_pairs[lab]
            # Fill columns left→right by ascending band index
            for c in range(n_cols):
                ax = axes[r][c]
                if c < len(pairs):
                    idx, xcol, ycol = pairs[c]

                    # find any present error flag cols for this pair; non-zero => error => exclude
                    err_cols = _present_error_cols(cols, corrected, idx) + _present_error_cols(cols, lab, idx)

                    df = _sample_pair_df(
                        con, p, xcol, ycol, max_points_per_pair, sentinel_at_or_below,
                        extra_zero_filters=err_cols if err_cols else None
                    )

                    if not df.empty:
                        x = df["x"].to_numpy()
                        y = df["y"].to_numpy()
                        ax.plot(x, y, ".", markersize=0.8, alpha=0.3)

                        # 1:1 line
                        if show_ones_line:
                            mn = float(np.nanmin(df[["x", "y"]].to_numpy()))
                            mx = float(np.nanmax(df[["x", "y"]].to_numpy()))
                            ax.plot([mn, mx], [mn, mx], linewidth=1.0, alpha=0.5, color="gray")

                        # regression (y ~ m x + b), draw and annotate
                        if x.size >= 2 and y.size >= 2:
                            m, b = _linreg(x, y)
                            if np.isfinite(m) and np.isfinite(b):
                                xx = np.linspace(np.nanmin(x), np.nanmax(x), 2)
                                yy = m * xx + b
                                ax.plot(xx, yy, "--", linewidth=1.0, alpha=0.9, color="red")
                                # compute R²
                                ss_res = float(np.nansum((y - (m*x + b))**2))
                                ss_tot = float(np.nansum((y - np.nanmean(y))**2))
                                r2 = 1 - ss_res/ss_tot if ss_tot > 0 else float("nan")
                                # equation text (top-left inside axes)
                                ax.text(
                                    0.02, 0.98,
                                    f"y = {m:.3f}x + {b:.3f}\nR² = {r2:.3f}   n={x.size:,}",
                                    transform=ax.transAxes,
                                    ha="left", va="top", fontsize=8,
                                    bbox=dict(facecolor="white", alpha=0.5, edgecolor="none", pad=1),
                                )
                        try:
                            r_val = float(np.corrcoef(x, y)[0, 1])
                        except Exception:
                            r_val = float("nan")
                    else:
                        r_val = float("nan")

                    # -------- labels using real wavelengths/FWHMs --------
                    sensor_text = _sensor_band_text(lab, idx)
                    corrected_text = _corrected_band_text(corrected, idx)
                    ax.set_title(f"{sensor_text} vs {corrected_text}  r={r_val:.2f}", fontsize=9)
                    ax.set_xlabel(corrected_text, fontsize=8)
                    if c == 0:
                        ax.set_ylabel(sensor_text, fontsize=8)
                    else:
                        ax.set_ylabel("")
                else:
                    ax.axis("off")

        fig.suptitle(f"{p.stem} — Sensors by row; bands left→right vs {corrected}", y=0.995, fontsize=12)
        fig.tight_layout()

        out_png = (Path(output_root) / f"{p.stem}__BY_SENSOR_vs_{corrected}.png")
        fig.savefig(out_png, dpi=200, bbox_inches="tight")
        plt.close(fig)
        outputs.append(out_png)

        if dropped_bands and log:
            print(f"[INFO] {p.name}: dropped {len(dropped_bands)} band(s) as palette-like (≥ {palette_col_frac_threshold:.2f}).")
        if log:
            print(f"[OK] {out_png}")

    con.close()
    return outputs


# ---------------------------------------------------------------------------
# Notebook example (commented)
# ---------------------------------------------------------------------------
# if False:
#     plots = make_all_sensor_panels(
#         "full_extracted_pixels_merged",
#     )
#     print(len(plots))


In [13]:
#from sensor_panel_plots_notebook import make_all_sensor_panels
plots = make_all_sensor_panels("full_extracted_pixels_merged")
len(plots), plots[:3]


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[OK] full_extracted_pixels_merged_compare_panels/NEON_D13_NIWO_DP1_L020-1_20230815_merged__BY_SENSOR_vs_NEON_directional_BRDFTopo.png


(1,
 [PosixPath('full_extracted_pixels_merged_compare_panels/NEON_D13_NIWO_DP1_L020-1_20230815_merged__BY_SENSOR_vs_NEON_directional_BRDFTopo.png')])

In [10]:
from pathlib import Path
import re
import duckdb

# --- patterns ---
# old broad prefix matcher (kept as fallback)
_GROUP_RE = re.compile(r'^(?P<prefix>.+?_L\d{3}-\d_\d{8}_)')

# canonical key: NEON_<domain>_<site>_DP1[.product]?_L###-#_<YYYYMMDD>_
_CANON_RE = re.compile(
    r'^(?P<site>NEON_[A-Z0-9]+_[A-Z0-9]+_DP1)(?:\.\d+\.\d+)?_(?P<tile>L\d{3}-\d)_(?P<date>\d{8})_'
)

def _canonical_group_key(fname: str) -> str | None:
    m = _CANON_RE.match(fname)
    if m:
        return f"{m.group('site')}_{m.group('tile')}_{m.group('date')}_"
    m2 = _GROUP_RE.match(fname)
    return m2.group('prefix') if m2 else None

# sensor/source label recognizer (order matters)
def _label_from_name(name: str) -> str:
    s = name
    if "resampled_Landsat_9_OLI-2" in s:   return "Landsat_9_OLI-2"
    if "resampled_Landsat_8_OLI" in s:     return "Landsat_8_OLI"
    if "resampled_Landsat_7_ETM+" in s:    return "Landsat_7_ETM+"
    if "resampled_Landsat_5_TM" in s:      return "Landsat_5_TM"
    if "resampled_MicaSense-to-match_OLI_and_OLI-2" in s: return "MicaSense_to-match_OLI_and_OLI-2"
    if "resampled_MicaSense-to-match_TM_and_ETM+" in s:   return "MicaSense_to-match_TM_and_ETM+"
    if "resampled_MicaSense_envi" in s or "resampled_MicaSense" in s: return "MicaSense"
    if "directional_reflectance_brdfandtopo_corrected_envi" in s: return "NEON_directional_BRDFTopo"
    if "reflectance_brdfandtopo_corrected_envi" in s:              return "NEON_BRDFTopo"
    return "Unknown"

# band column pattern
_BAND_RE = re.compile(r"^(ENVI|Masked|Original)_band_(\d+)$")

def _quote_ident(col: str) -> str:
    return '"' + col.replace('"', '""') + '"'

def _band_cols_duckdb(con: duckdb.DuckDBPyConnection, parquet_path: Path) -> list[str]:
    rows = con.execute(f"DESCRIBE SELECT * FROM read_parquet('{parquet_path.as_posix()}')").fetchall()
    names = [r[0] for r in rows]
    return [n for n in names if _BAND_RE.match(n)]

def merge_matching_parquet_sets(
    input_dir,
    output_dir=None,
    join_on=("Pixel_ID",),
    keep_meta=("Pixel_ID","Pixel_X","Pixel_Y","CRS"),
    expected_big=426,
    verbose=True
):
    """
    Merge sets of parquet files that share the canonical prefix:
      NEON_<domain>_<site>_DP1[.product]?_L###-#_<YYYYMMDD>_*

    - Picks a 426-band file as base when available.
    - LEFT JOINs all other files on Pixel_ID (fallback Pixel_X,Pixel_Y).
    - Renames bands to <LABEL>_band_<n> and SELECTs them (fixes previous t0.*-only bug).
    - Writes one merged parquet per set into a sibling "<dir>_merged/" (or custom output_dir).
    """
    input_dir = Path(input_dir)
    if output_dir is None:
        output_dir = input_dir.parent / f"{input_dir.name}_merged"
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    con = duckdb.connect()
    files = sorted(input_dir.glob("*.parquet"))

    # group files by canonical key
    groups: dict[str, list[Path]] = {}
    for f in files:
        key = _canonical_group_key(f.name)
        if not key:
            if verbose:
                print(f"[SKIP] Unrecognized name (no group key): {f.name}")
            continue
        groups.setdefault(key, []).append(f)

    if not groups:
        print(f"[INFO] No matching sets found under: {input_dir}")
        con.close()
        return []

    written = []

    for key, paths in sorted(groups.items()):
        # pick base (prefer largest band count)
        band_counts = {}
        for p in paths:
            try:
                band_counts[p] = len(_band_cols_duckdb(con, p))
            except Exception:
                band_counts[p] = 0

        base_path = sorted(paths, key=lambda p: (-band_counts.get(p, 0), p.name))[0]
        base_label = _label_from_name(base_path.name)
        base_bands = _band_cols_duckdb(con, base_path)

        # determine join cols
        rows = con.execute(f"DESCRIBE SELECT * FROM read_parquet('{base_path.as_posix()}')").fetchall()
        base_cols = {r[0] for r in rows}
        join_cols = list(join_on)
        if not all(c in base_cols for c in join_cols):
            if {"Pixel_X","Pixel_Y"}.issubset(base_cols):
                join_cols = ["Pixel_X","Pixel_Y"]
            else:
                raise RuntimeError(f"Base {base_path.name} missing join cols; has: {sorted(list(base_cols))[:20]}")

        # build base CTE (rename its bands)
        base_select = []
        for c in keep_meta:
            if c in base_cols:
                base_select.append(_quote_ident(c))
        for b in base_bands:
            m = _BAND_RE.match(b)
            idx = m.group(2) if m else b
            newname = f"{base_label}_band_{idx}"
            base_select.append(f"{_quote_ident(b)} AS {_quote_ident(newname)}")

        base_cte_name = "t0"
        base_cte = f"""{base_cte_name} AS (
            SELECT {", ".join(base_select)}
            FROM read_parquet('{base_path.as_posix()}')
        )"""

        # other CTEs + joins
        ctes = [base_cte]
        join_clauses = []
        select_list = [f"{base_cte_name}.*"]  # start with all base cols

        t_idx = 1
        for p in sorted(paths):
            if p == base_path:
                continue

            label = _label_from_name(p.name)
            bcols = _band_cols_duckdb(con, p)
            if not bcols:
                if verbose:
                    print(f"[WARN] {p.name}: no band columns, skipping.")
                continue

            rows = con.execute(f"DESCRIBE SELECT * FROM read_parquet('{p.as_posix()}')").fetchall()
            cols = {r[0] for r in rows}

            # ensure join cols exist
            if not all(c in cols for c in join_cols):
                if verbose:
                    print(f"[WARN] {p.name}: missing join columns {join_cols}, skipping.")
                continue

            # project join cols + renamed bands
            sel_parts = [ _quote_ident(c) for c in join_cols ]
            renamed_cols = []  # keep the new names so we can SELECT them later
            for b in bcols:
                m = _BAND_RE.match(b)
                idx = m.group(2) if m else b
                newname = f"{label}_band_{idx}"
                sel_parts.append(f"{_quote_ident(b)} AS {_quote_ident(newname)}")
                renamed_cols.append(newname)

            ti = f"t{t_idx}"
            ctes.append(f"""{ti} AS (
                SELECT {", ".join(sel_parts)}
                FROM read_parquet('{p.as_posix()}')
            )""")

            # LEFT JOIN USING (join_cols)
            using_clause = ", ".join(_quote_ident(c) for c in join_cols)
            join_clauses.append(f"LEFT JOIN {ti} USING ({using_clause})")

            # **FIX**: actually SELECT the joined sensor columns
            select_list.extend(f'{ti}.{_quote_ident(c)}' for c in renamed_cols)

            t_idx += 1

        # write output
        output_name = key.rstrip("_") + "_merged.parquet"
        out_path = output_dir / output_name

        with_clause = "WITH " + ",\n".join(ctes)
        joins = "\n".join(join_clauses)
        final_sql = f"""{with_clause}
        SELECT {", ".join(select_list)}
        FROM {base_cte_name}
        {joins}
        """

        con.execute(f"COPY ({final_sql}) TO '{out_path.as_posix()}' (FORMAT PARQUET)")

        if verbose:
            big_count = len(base_bands)
            if big_count != expected_big:
                print(f"[INFO] {base_path.name}: base bands={big_count} (expected ~{expected_big})")
            print(f"[OK] Wrote {out_path} from {len(paths)} files; join keys: {join_cols}")

        written.append(out_path)

    con.close()
    return written

outs = merge_matching_parquet_sets("full_extracted_pixels", verbose=True)

# Preview the merged columns for one output:
import duckdb, pathlib
con = duckdb.connect()
merged = pathlib.Path("full_extracted_pixels_merged")
p = sorted(merged.glob("*.parquet"))[0]
con.sql(f"DESCRIBE SELECT * FROM read_parquet('{p.as_posix()}')").df()[["column_name","column_type"]]



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[OK] Wrote full_extracted_pixels_merged/NEON_D13_NIWO_DP1_L020-1_20230815_merged.parquet from 8 files; join keys: ['Pixel_ID']


,column_name,column_type
0,Pixel_ID,BIGINT
1,Pixel_X,DOUBLE
2,Pixel_Y,DOUBLE
3,CRS,VARCHAR
4,NEON_directional_BRDFTopo_band_1,FLOAT
...,...,...
470,MicaSense_band_6,FLOAT
471,MicaSense_band_7,FLOAT
472,MicaSense_band_8,FLOAT
473,MicaSense_band_9,FLOAT


In [11]:
import duckdb

path = "full_extracted_pixels_merged/NEON_D13_NIWO_DP1_L020-1_20230815_merged.parquet"

duckdb.sql(f"SELECT * FROM read_parquet('{path}') LIMIT 5").df()



,Pixel_ID,Pixel_X,Pixel_Y,CRS,NEON_directional_BRDFTopo_band_1,NEON_directional_BRDFTopo_band_2,NEON_directional_BRDFTopo_band_3,NEON_directional_BRDFTopo_band_4,NEON_directional_BRDFTopo_band_5,NEON_directional_BRDFTopo_band_6,NEON_directional_BRDFTopo_band_7,NEON_directional_BRDFTopo_band_8,NEON_directional_BRDFTopo_band_9,NEON_directional_BRDFTopo_band_10,NEON_directional_BRDFTopo_band_11,NEON_directional_BRDFTopo_band_12,NEON_directional_BRDFTopo_band_13,NEON_directional_BRDFTopo_band_14,NEON_directional_BRDFTopo_band_15,NEON_directional_BRDFTopo_band_16,NEON_directional_BRDFTopo_band_17,NEON_directional_BRDFTopo_band_18,NEON_directional_BRDFTopo_band_19,NEON_directional_BRDFTopo_band_20,NEON_directional_BRDFTopo_band_21,NEON_directional_BRDFTopo_band_22,NEON_directional_BRDFTopo_band_23,NEON_directional_BRDFTopo_band_24,NEON_directional_BRDFTopo_band_25,NEON_directional_BRDFTopo_band_26,NEON_directional_BRDFTopo_band_27,NEON_directional_BRDFTopo_band_28,NEON_directional_BRDFTopo_band_29,NEON_directional_BRDFTopo_band_30,NEON_directional_BRDFTopo_band_31,NEON_directional_BRDFTopo_band_32,NEON_directional_BRDFTopo_band_33,NEON_directional_BRDFTopo_band_34,NEON_directional_BRDFTopo_band_35,NEON_directional_BRDFTopo_band_36,NEON_directional_BRDFTopo_band_37,NEON_directional_BRDFTopo_band_38,NEON_directional_BRDFTopo_band_39,NEON_directional_BRDFTopo_band_40,NEON_directional_BRDFTopo_band_41,NEON_directional_BRDFTopo_band_42,NEON_directional_BRDFTopo_band_43,NEON_directional_BRDFTopo_band_44,NEON_directional_BRDFTopo_band_45,NEON_directional_BRDFTopo_band_46,NEON_directional_BRDFTopo_band_47,NEON_directional_BRDFTopo_band_48,NEON_directional_BRDFTopo_band_49,NEON_directional_BRDFTopo_band_50,NEON_directional_BRDFTopo_band_51,NEON_directional_BRDFTopo_band_52,NEON_directional_BRDFTopo_band_53,NEON_directional_BRDFTopo_band_54,NEON_directional_BRDFTopo_band_55,NEON_directional_BRDFTopo_band_56,NEON_directional_BRDFTopo_band_57,NEON_directional_BRDFTopo_band_58,NEON_directional_BRDFTopo_band_59,NEON_directional_BRDFTopo_band_60,NEON_directional_BRDFTopo_band_61,NEON_directional_BRDFTopo_band_62,NEON_directional_BRDFTopo_band_63,NEON_directional_BRDFTopo_band_64,NEON_directional_BRDFTopo_band_65,NEON_directional_BRDFTopo_band_66,NEON_directional_BRDFTopo_band_67,NEON_directional_BRDFTopo_band_68,NEON_directional_BRDFTopo_band_69,NEON_directional_BRDFTopo_band_70,NEON_directional_BRDFTopo_band_71,NEON_directional_BRDFTopo_band_72,NEON_directional_BRDFTopo_band_73,NEON_directional_BRDFTopo_band_74,NEON_directional_BRDFTopo_band_75,NEON_directional_BRDFTopo_band_76,NEON_directional_BRDFTopo_band_77,NEON_directional_BRDFTopo_band_78,NEON_directional_BRDFTopo_band_79,NEON_directional_BRDFTopo_band_80,NEON_directional_BRDFTopo_band_81,NEON_directional_BRDFTopo_band_82,NEON_directional_BRDFTopo_band_83,NEON_directional_BRDFTopo_band_84,NEON_directional_BRDFTopo_band_85,NEON_directional_BRDFTopo_band_86,NEON_directional_BRDFTopo_band_87,NEON_directional_BRDFTopo_band_88,NEON_directional_BRDFTopo_band_89,NEON_directional_BRDFTopo_band_90,NEON_directional_BRDFTopo_band_91,NEON_directional_BRDFTopo_band_92,NEON_directional_BRDFTopo_band_93,NEON_directional_BRDFTopo_band_94,NEON_directional_BRDFTopo_band_95,NEON_directional_BRDFTopo_band_96,NEON_directional_BRDFTopo_band_97,NEON_directional_BRDFTopo_band_98,NEON_directional_BRDFTopo_band_99,NEON_directional_BRDFTopo_band_100,NEON_directional_BRDFTopo_band_101,NEON_directional_BRDFTopo_band_102,NEON_directional_BRDFTopo_band_103,NEON_directional_BRDFTopo_band_104,NEON_directional_BRDFTopo_band_105,NEON_directional_BRDFTopo_band_106,NEON_directional_BRDFTopo_band_107,NEON_directional_BRDFTopo_band_108,NEON_directional_BRDFTopo_band_109,NEON_directional_BRDFTopo_band_110,NEON_directional_BRDFTopo_band_111,NEON_directional_BRDFTopo_band_112,NEON_directional_BRDFTopo_band_113,NEON_directional_BRDFTopo_band_114,NEON_directional_BRDFTopo_band_115,NEON_directional_BRDFTopo_band_116,NEON_directional_B